In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
model_data_labels = transcript[transcript['event'] == 'offer received'][['person','value','time']].reset_index(drop = True)

In [3]:
model_data_labels['offer_id'] = model_data_labels['value'].apply(lambda x: x.get('offer id'))
model_data_labels['label'] = 0
model_data_labels = model_data_labels.drop('value',axis = 1)

In [4]:
model_data_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 4 columns):
person      76277 non-null object
time        76277 non-null int64
offer_id    76277 non-null object
label       76277 non-null int64
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [5]:
model_data_labels[['person','offer_id']].drop_duplicates().count()

person      63288
offer_id    63288
dtype: int64

In [6]:
portfolio.rename(columns = {'id' : 'offer_id'},inplace = True)
model_data_labels = pd.merge(model_data_labels,portfolio[['offer_id','duration']],on = 'offer_id')

In [7]:
model_data_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76277 entries, 0 to 76276
Data columns (total 5 columns):
person      76277 non-null object
time        76277 non-null int64
offer_id    76277 non-null object
label       76277 non-null int64
duration    76277 non-null int64
dtypes: int64(3), object(2)
memory usage: 3.5+ MB


In [8]:
offer_viewed = transcript[transcript['event'] == 'offer viewed'][['person','value','time']].copy().reset_index(drop = True)
offer_completed= transcript[transcript['event'] == 'offer completed'][['person','value','time']].copy().reset_index(drop = True)
offer_viewed['offer_id'] = offer_viewed['value'].apply(lambda x: x.get('offer id'))
offer_completed['offer_id'] = offer_completed['value'].apply(lambda x: x.get('offer_id'))

In [9]:
offer_completed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33579 entries, 0 to 33578
Data columns (total 4 columns):
person      33579 non-null object
value       33579 non-null object
time        33579 non-null int64
offer_id    33579 non-null object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [10]:
offer_viewed_completed = pd.merge(offer_viewed[['person','offer_id','time']],
                                  offer_completed[['person','offer_id','time']],on = ['person','offer_id'])

In [11]:
offer_viewed_completed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37952 entries, 0 to 37951
Data columns (total 4 columns):
person      37952 non-null object
offer_id    37952 non-null object
time_x      37952 non-null int64
time_y      37952 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.4+ MB


In [12]:
offer_viewed_completed = offer_viewed_completed[offer_viewed_completed['time_x'] <= offer_viewed_completed['time_y']].copy().reset_index(drop = True)

In [13]:
offer_viewed_completed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29098 entries, 0 to 29097
Data columns (total 4 columns):
person      29098 non-null object
offer_id    29098 non-null object
time_x      29098 non-null int64
time_y      29098 non-null int64
dtypes: int64(2), object(2)
memory usage: 909.4+ KB


In [14]:
offer_viewed_completed = offer_viewed_completed.rename(columns = {'person':'person_right','offer_id' : 'offer_id_right'})
model_data_labels_completed = pd.merge(model_data_labels,offer_viewed_completed,how = 'inner',left_on = ['person','offer_id']
                            ,right_on = ['person_right','offer_id_right'])

In [15]:
model_data_labels_completed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45075 entries, 0 to 45074
Data columns (total 9 columns):
person            45075 non-null object
time              45075 non-null int64
offer_id          45075 non-null object
label             45075 non-null int64
duration          45075 non-null int64
person_right      45075 non-null object
offer_id_right    45075 non-null object
time_x            45075 non-null int64
time_y            45075 non-null int64
dtypes: int64(5), object(4)
memory usage: 3.4+ MB


In [16]:
model_data_labels_completed['time_diff'] = model_data_labels_completed['time_y'] - model_data_labels_completed['time']
model_data_labels_completed = model_data_labels_completed[model_data_labels_completed['time_diff'] <= model_data_labels_completed['duration'] * 24].copy().reset_index(drop = True)

In [17]:
model_data_labels_completed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36416 entries, 0 to 36415
Data columns (total 10 columns):
person            36416 non-null object
time              36416 non-null int64
offer_id          36416 non-null object
label             36416 non-null int64
duration          36416 non-null int64
person_right      36416 non-null object
offer_id_right    36416 non-null object
time_x            36416 non-null int64
time_y            36416 non-null int64
time_diff         36416 non-null int64
dtypes: int64(6), object(4)
memory usage: 2.8+ MB


In [18]:
model_data_labels_completed = model_data_labels_completed.groupby(['person','offer_id']
                                                                  ,as_index = False)['time'].min()

In [19]:
model_data_labels_completed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21372 entries, 0 to 21371
Data columns (total 3 columns):
person      21372 non-null object
offer_id    21372 non-null object
time        21372 non-null int64
dtypes: int64(1), object(2)
memory usage: 667.9+ KB


In [20]:
model_data_labels_completed['label_completed'] = 1
model_data_labels = pd.merge(model_data_labels,model_data_labels_completed,how = 'left',on = ['person','offer_id','time'])

In [21]:
model_data_labels['label'] = np.where(model_data_labels['label_completed'] ==1,1,0)

In [22]:
model_data_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76277 entries, 0 to 76276
Data columns (total 6 columns):
person             76277 non-null object
time               76277 non-null int64
offer_id           76277 non-null object
label              76277 non-null int64
duration           76277 non-null int64
label_completed    21372 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 4.1+ MB


In [23]:
model_data_labels0 = model_data_labels[model_data_labels['label'] == 0].copy().reset_index(drop = True)

In [24]:
model_data_labels0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54905 entries, 0 to 54904
Data columns (total 6 columns):
person             54905 non-null object
time               54905 non-null int64
offer_id           54905 non-null object
label              54905 non-null int64
duration           54905 non-null int64
label_completed    0 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.5+ MB


In [25]:
model_data_labels0_not_completed = model_data_labels0[model_data_labels0['time'] > 714 - 
                                                      model_data_labels0['duration'] * 24].copy()\
                                                                                        .reset_index(drop = True)

In [26]:
model_data_labels0_not_completed_unique = model_data_labels0_not_completed.\
groupby(['person','offer_id','duration'],as_index = False)['time'].min()

In [27]:
model_data_labels0_not_completed_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7278 entries, 0 to 7277
Data columns (total 4 columns):
person      7278 non-null object
offer_id    7278 non-null object
duration    7278 non-null int64
time        7278 non-null int64
dtypes: int64(2), object(2)
memory usage: 284.3+ KB


In [28]:
offer_completed = offer_completed.rename(columns = {'person':'person_right',
                                                    'offer_id' : 'offer_id_right',
                                                    'time' : 'time_right'})
model_data_labels0_not_completed_unique = pd.merge(model_data_labels0_not_completed_unique,
                                            offer_completed,
                                            how = 'left',
                                            left_on = ['person','offer_id'],
                                            right_on = ['person_right','offer_id_right']
                                           )

In [29]:
model_data_labels0_not_completed_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8744 entries, 0 to 8743
Data columns (total 8 columns):
person            8744 non-null object
offer_id          8744 non-null object
duration          8744 non-null int64
time              8744 non-null int64
person_right      4539 non-null object
value             4539 non-null object
time_right        4539 non-null float64
offer_id_right    4539 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 614.8+ KB


In [30]:
model_data_labels0_not_completed_unique = model_data_labels0_not_completed_unique[
    model_data_labels0_not_completed_unique['person_right'].isnull()
].copy().reset_index(drop = True)

In [31]:
model_data_labels0_not_completed_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4205 entries, 0 to 4204
Data columns (total 8 columns):
person            4205 non-null object
offer_id          4205 non-null object
duration          4205 non-null int64
time              4205 non-null int64
person_right      0 non-null object
value             0 non-null object
time_right        0 non-null float64
offer_id_right    0 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 262.9+ KB


In [32]:
model_data_labels0_not_completed_unique[['person','offer_id']].drop_duplicates().shape

(4205, 2)

In [33]:
model_data_labels0_not_completed_unique = model_data_labels0_not_completed_unique[['person','offer_id']].\
rename(columns = {'person' : 'person_right',
                  'offer_id' : 'offer_id_right'})
model_data_labels0 = pd.merge(model_data_labels0,
                              model_data_labels0_not_completed_unique[['person_right','offer_id_right']],
                              how = 'left', left_on = ['person','offer_id'],
                              right_on = ['person_right','offer_id_right'])

model_data_labels0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54905 entries, 0 to 54904
Data columns (total 8 columns):
person             54905 non-null object
time               54905 non-null int64
offer_id           54905 non-null object
label              54905 non-null int64
duration           54905 non-null int64
label_completed    0 non-null float64
person_right       5327 non-null object
offer_id_right     5327 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 3.8+ MB


In [34]:
model_data_labels0 = model_data_labels0[(model_data_labels0['person_right'].isnull())].reset_index(drop = True).copy()
model_data_labels0 = model_data_labels0[['person','offer_id','time']]

In [35]:
# model_data_labels0_not_completed_unique['time_diff'] = model_data_labels0_not_completed_unique['time_right'] - model_data_labels0_not_completed_unique['time']
# model_data_labels0_not_completed_unique = model_data_labels0_not_completed_unique[model_data_labels0_not_completed_unique['time_diff'] <= model_data_labels0_not_completed_unique['duration'] * 24].copy().reset_index(drop = True)


In [36]:
model_data_labels0_unique = model_data_labels0.groupby(['person','offer_id'],as_index = False)['time'].max()
model_data_labels0_unique['label'] = 0

In [37]:
final_data = pd.concat([model_data_labels0_unique,
                        model_data_labels[model_data_labels['label'] == 1][['person','offer_id','time','label']]],
                       ignore_index = True)

In [38]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64481 entries, 0 to 64480
Data columns (total 4 columns):
person      64481 non-null object
offer_id    64481 non-null object
time        64481 non-null int64
label       64481 non-null int64
dtypes: int64(2), object(2)
memory usage: 2.0+ MB


In [39]:
final_data = final_data.drop_duplicates(subset = ['person','offer_id'],keep = 'last').reset_index(drop = True)

In [40]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59083 entries, 0 to 59082
Data columns (total 4 columns):
person      59083 non-null object
offer_id    59083 non-null object
time        59083 non-null int64
label       59083 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


In [41]:
63288 - 4205

59083

In [42]:
final_data['label'].value_counts()

0    37711
1    21372
Name: label, dtype: int64

In [43]:
final_data['time'].max()

576

In [44]:
576/24

24.0

In [45]:
final_data['day'] = final_data['time']/24
final_data[final_data['day'] > 20].shape

(18035, 5)

In [46]:
22240/63288

0.35140942990772345

In [47]:
from sklearn.model_selection import train_test_split
y=final_data['label']
df_train,df_test=train_test_split(final_data, test_size=0.33,random_state=0,stratify = y)

In [48]:
df_test.head()

,person,offer_id,time,label,day
45588,a5675e02cac04aefb87d21551a3dfa8c,fafdcd668e3743c1bb461111dcafc2a4,408,1,17.0
46913,5c29ee406ced442ca7fe5523b4160412,fafdcd668e3743c1bb461111dcafc2a4,576,1,24.0
15995,6cde0dadd1154cf89694618ce6f249da,0b1e1539f2cc45b7b9fa7c272da2e1d7,576,0,24.0
5703,28d2a4892f5b42c7afd7583c26fbfe21,5a8bc65990b245e5a138643cd4eb9837,336,0,14.0
20222,88f63e6b5de440cfa91a609403fc6993,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,0,21.0


In [49]:
df_test['label'].value_counts()

0    12445
1     7053
Name: label, dtype: int64

In [2]:
12445/(12445 + 7053)

0.6382705918555749

In [ ]:
final_data.to_csv('final_labelled_data.csv',encoding = 'utf-8',index = False)